<a href="https://colab.research.google.com/github/kaisacht/Project1/blob/main/Resnet1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import numpy as np
import pandas as pd


In [ ]:
np.random.seed(813306)
print(np)

<module 'numpy' from '/usr/local/lib/python3.8/dist-packages/numpy/__init__.py'>


In [ ]:
def build_resnet(input_shape, n_feature_maps, nb_classes):
    print ('build conv_x')
    x = keras.layers.Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv2D(n_feature_maps, 8, 1, padding='same')(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, 1, 1,padding='same')(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
    
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    full = keras.layers.GlobalAveragePooling2D()(y)
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    print ('        -- model was built.')
    return x, out

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 20_day_TRAIN.txt to 20_day_TRAIN.txt


In [ ]:
df = pd.read_csv("20_day_TEST.txt")

In [ ]:
print(df)

     3   29.17   29.03   28.87   28.81    28.8   29.11   29.22   29.32  \
0    3   29.44   29.45   29.53   29.73   29.43   29.65   30.64   31.03   
1    1   30.13   29.41   29.22   29.25   29.20   28.93   28.83   28.96   
2    3   27.41   27.65   27.55   26.60   27.03   27.13   26.71   26.73   
3    3   27.56   27.75   27.90   28.30   28.39   28.22   27.99   28.04   
4    3   29.90   30.30   30.53   30.29   30.48   30.58   30.57   30.53   
..  ..     ...     ...     ...     ...     ...     ...     ...     ...   
194  1  261.60  268.41  267.22  265.94  269.61  264.63  252.53  241.53   
195  3  254.61  254.74  262.40  265.02  263.29  262.18  252.04  248.11   
196  1  275.84  272.38  276.61  280.17  278.68  279.32  277.61  286.94   
197  1  263.85  260.66  261.33  255.41  254.47  251.94  253.22  254.79   
198  3  234.50  234.73  234.41  232.73  234.66  244.98  244.10  246.08   

      29.52  ...   29.78   29.53   29.97   29.89   29.62    29.4   29.91  \
0     31.01  ...   30.69   30.51   

In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y

In [ ]:
x_train, y_train = readucr('20_day_TRAIN.txt')
x_test, y_test = readucr('20_day_TEST.txt')

In [ ]:
print(x_test,y_test)

[[[[ 0.76280165]]

  [[ 0.75250015]]

  [[ 0.74072701]]

  ...

  [[ 0.80989424]]

  [[ 0.81136588]]

  [[ 0.77972555]]]


 [[[ 0.78266884]]

  [[ 0.78340466]]

  [[ 0.78929123]]

  ...

  [[ 0.83417636]]

  [[ 0.84962861]]

  [[ 0.85110025]]]


 [[[ 0.83344053]]

  [[ 0.78046137]]

  [[ 0.76648076]]

  ...

  [[ 0.67082395]]

  [[ 0.65684334]]

  [[ 0.6590508 ]]]


 ...


 [[[18.9133144 ]]

  [[18.65872011]]

  [[18.96997267]]

  ...

  [[18.85959942]]

  [[18.81618595]]

  [[18.33495858]]]


 [[[18.03106424]]

  [[17.79633713]]

  [[17.84563718]]

  ...

  [[16.09732493]]

  [[15.9229352 ]]

  [[16.04802488]]]


 [[[15.87142768]]

  [[15.88835158]]

  [[15.86480529]]

  ...

  [[15.89865309]]

  [[15.87142768]]

  [[16.37178641]]]] [1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1.
 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 

In [ ]:

nb_epochs = 1500

In [ ]:
flist  = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr('20_day_TRAIN.txt')
    x_test, y_test = readucr('20_day_TEST.txt')
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0]/10, 16)
     
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
     
     
    Y_train = keras.utils.to_categorical(y_train, nb_classes)
    Y_test = keras.utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
      
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))
     
     
    x , y = build_resnet(x_train.shape[1:], 64, nb_classes)
    model = keras.models.Model(inputs=x, outputs=y)
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
      
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    log = pd.DataFrame(hist.history)

build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
        -- model was built.
Epoch 1/1500
13/13 [==============================] - 5s 266ms/step - loss: 0.9655 - accuracy: 0.5396 - val_loss: 0.7059 - val_accuracy: 0.4200 - lr: 0.0010
Epoch 2/1500
13/13 [==============================] - 3s 232ms/step - loss: 0.7655 - accuracy: 0.5446 - val_loss: 0.8211 - val_accuracy: 0.5300 - lr: 0.0010
Epoch 3/1500
13/13 [==============================] - 3s 232ms/step - loss: 0.6488 - accuracy: 0.6238 - val_loss: 1.3477 - val_accuracy: 0.3650 - lr: 0.0010
Epoch 4/1500
13/13 [==============================] - 3s 233ms/step - loss: 0.6814 - accuracy: 0.6040 - val_loss: 0.9785 - val_accuracy: 0.3750 - lr: 0.0010
Epoch 5/1500
13/13 [==============================] - 3s 234ms/step - loss: 0.6580 - accuracy: 0.6238 - val_loss: 0.8341 - val_accuracy: 0.4500 - lr: 0.001

In [ ]:
print(log)

          loss  accuracy  val_loss  val_accuracy      lr
0     0.965538  0.539604  0.705905         0.420  0.0010
1     0.765481  0.544554  0.821052         0.530  0.0010
2     0.648820  0.623762  1.347652         0.365  0.0010
3     0.681370  0.603960  0.978529         0.375  0.0010
4     0.657970  0.623762  0.834113         0.450  0.0010
...        ...       ...       ...           ...     ...
1495  0.045887  0.985148  1.546856         0.775  0.0001
1496  0.044736  0.980198  3.528858         0.715  0.0001
1497  0.056838  0.980198  4.319671         0.700  0.0001
1498  0.048676  0.980198  4.166442         0.690  0.0001
1499  0.090153  0.955446  3.289588         0.730  0.0001

[1500 rows x 5 columns]


In [ ]:
log_np = np.array(log)
log_np
min_index = (np.argmin(log_np, axis=0))
print(log_np[min_index[0]])


[1.76958200e-02 1.00000000e+00 1.36114919e+00 7.74999976e-01
 9.99999975e-05]


In [ ]:
print(y_test)

[20. 12.  5.  5.  6. 31.  7.  0. 19. 24.  8. 31. 25.  3. 21. 29. 13. 23.
  3.  3. 20. 13. 22. 30. 30. 26.  4.  3. 32.  2.  3.  6. 27. 12.  0. 13.
  0. 27. 31. 15.  9. 11. 31.  3. 21.  2. 19. 20.  2. 31.  2.  9.  5. 30.
 28. 12. 25. 23. 22. 13. 26. 28. 20.  1. 14. 14. 27. 17. 30. 32. 13.  2.
  9. 30.  3. 26. 32. 10. 23. 20.  2.  0. 26. 12. 13.  0. 13. 20.  5.  5.
  7. 24. 20.  5.  8.  1. 17.  2. 18. 14. 19. 27. 26. 31. 27.  2. 26. 21.
 22.  8.  6. 26. 28. 30. 17.  0.  0.  0. 30. 31. 27.  2.  5.  9. 20. 16.
  2. 24. 31.  9.  1.  1. 24. 11. 32.  2.  6.  8. 10.  6.  3. 13. 17. 24.
  2. 32.  0.  5. 24.  5. 10. 19. 31.  8.  8.  1.  5.  8.  0. 11. 11.  0.
  8. 25.  0. 23.  4.  6.  0. 25.  0. 27. 24. 22. 31. 20.  1. 10. 31.  9.
  2. 28.  0.  2.  2. 13. 22.  6. 20. 15. 21.  5.  8. 30. 29.  5.  5. 31.
 22. 22.]


# Mục mới